In [29]:
import numpy as np
import pandas as pd
from   scipy.stats import norm
from src.config import BLD,SRC


data =  pd.read_stata( SRC /"replication_files"/"original_data"/"decisions_data.dta" )
data.loc[:,"pb"] = data["workdone1"] / 10
data

,wid,rowid,unixtime,decisiondate,decisiondatenum,workdate,workdatenum,workdatetrue,workdatetrueadded,workdatediff,...,session,left,leftdate,leftdatenum,timespent,payment,randomkey,leftduring,first,pb
0,1.0,12547,2012-11-02 21:35:30,2012-11-02,5.0,2012-11-07,6.0,2012-11-07,2012-10-17 10:58:13,0.0,...,1,0.0,NaT,NaN,27.274218,110.30,810227634,0.0,0.0,1.0
1,1.0,9273,2012-10-29 16:50:01,2012-10-29,4.0,2012-10-29,4.0,2012-10-29,2012-10-17 10:58:13,0.0,...,1,0.0,NaT,NaN,27.274218,110.30,810227634,0.0,0.0,0.0
2,1.0,12513,2012-11-02 21:26:11,2012-11-02,5.0,2012-11-07,6.0,2012-11-07,2012-10-17 10:58:13,0.0,...,1,0.0,NaT,NaN,27.274218,110.30,810227634,0.0,0.0,0.0
3,1.0,14827,2012-11-07 10:11:29,2012-11-07,6.0,2012-11-11,7.0,2012-11-11,2012-10-17 10:58:13,0.0,...,1,0.0,NaT,NaN,27.274218,110.30,810227634,0.0,0.0,1.0
4,1.0,9271,2012-10-29 16:50:01,2012-10-29,4.0,2012-10-29,4.0,2012-10-29,2012-10-17 10:58:13,0.0,...,1,0.0,NaT,NaN,27.274218,110.30,810227634,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11400,100.0,12631,2012-11-02 22:10:32,2012-11-02,4.0,2012-11-13,6.0,2012-11-13,2012-10-19 10:58:21,0.0,...,4,0.0,NaT,NaN,44.584381,26.23,237278484,0.0,0.0,1.0
11401,100.0,2509,2012-10-19 11:25:45,2012-10-19,1.0,2012-10-24,2.0,2012-10-24,2012-10-19 10:58:21,0.0,...,4,0.0,NaT,NaN,44.584381,26.23,237278484,0.0,0.0,1.0
11402,100.0,12625,2012-11-02 22:00:44,2012-11-02,4.0,2012-11-08,5.0,2012-11-08,2012-10-19 10:58:21,0.0,...,4,0.0,NaT,NaN,44.584381,26.23,237278484,0.0,0.0,0.0
11403,100.0,9341,2012-10-29 18:16:50,2012-10-29,3.0,2012-11-13,6.0,2012-11-13,2012-10-19 10:58:21,0.0,...,4,0.0,NaT,NaN,44.584381,26.23,237278484,0.0,0.0,1.0


## Likelihood Fct

In [ ]:




def negloglike(params, args, spec):
    if spec != 4:
        #Drop alpha from params datafrme          # columns 1, 2, 3 do not estimate α  #PROJECTION BIAS
    else:
        # just leave args beta, betahat, delta, gamma, phi, alpha, sigma = params # dont drop 
    
    # predchoice is the predicted choice coming from the optimality condition of the subject
    # if we are estimating the projection bias parameter α (column 4) we need to modify the predicted choice
    
    predchoice = ((params.loc["phi", "value"]*(params.loc["delta", "value"]**data["netdistance"])*(params.loc["beta", "value"]**data["today"])*(params.loc["betahat", "value"]**data["prediction"])*data["wage"])**(1/(params.loc["gamma", "value"]-1)))-data["pb"]*params.loc["alpha", "value"]
    prob = (1-data["ind_effort10"]-data["ind_effort110"])*norm.pdf(data["effort"], predchoice, params.loc["sigma", "value"])+data["ind_effort10"]*(1 - norm.cdf((predchoice-data["effort"])/params.loc["sigma", "value"]))+data["ind_effort110"]*norm.cdf((predchoice-data["effort"])/params.loc["sigma", "value"]) 

    if spec==4: predchoice = predchoice -data["pb"]*params.loc["alpha"] ##PROEJECTION BIAS
        
    # prob is a 1x8049 vector containing the probability of observing the effort of an individual. If effort is 10 or 110 we apply a Tobit correction
    
    prob = (1-ind_effort10-ind_effort110)*norm.pdf(effort, predchoice, sigma)+ind_effort10*(1 - norm.cdf((predchoice-effort)/sigma))+ind_effort110*norm.cdf((predchoice-effort)/sigma)
            
    # we now look at the vector prob and add a small value close to zero if prob=0 or subtract a small value close to zero if prob=1. This is necessary to avoid problems when taking logs
        
    index_p0 = [i for i in range(0,len(prob)) if prob[i]==0] # vector containing the indexes when prob=0
    index_p1 = [i for i in range(0,len(prob)) if prob[i]==1] # vector containing the indexes when prob=1
    
    # use a for loop to change the values
    
    for i in index_p0:
        prob[i] = 1E-4
        
    for i in index_p1:
        prob[i] = 1 - 1E-4
    
    negll = - np.sum(np.log(prob)) # negative log likelihood
    
    return negll

In [ ]:
Define an auxiliary function.
# This function takes as input a dataset, a scalar and a specification argument and returns the relevant individual-level dataframe containing only observations for individual whose ID is equal to that scalar

def getind(dataset, wid, spec):
    dataset_ind = dataset[dataset.wid == wid]
    if spec == 2: dataset_ind = dataset_ind[dataset_ind.decisiondatenum < 4]  # second column specification
    if spec == 3: dataset_ind = dataset_ind[dataset_ind.decisiondatenum >= 4] # third column specification    
    return dataset_ind